# Novel dataset web scraper

Gathers novel information from novelupdates, i.e. http://www.novelupdates.com/.  
The data is then cleaned and arranged into a dataset.  
The dataset is finally saved as a csv file.

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from scraper import NovelScraper

# Display all columns when showing dataframes
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
debug = True
scraper = NovelScraper(debug=debug)
novel_info = scraper.parse_all_novels()

Debug run, running with: 1
Processing novel: 27232
Processing novel: 29224
Processing novel: 2765
Processing novel: 9120
Processing novel: 11764
Processing novel: 14233
Processing novel: 29057
Processing novel: 14
Processing novel: 30064
Processing novel: 29462
Processing novel: 29338
Processing novel: 17132
Processing novel: 28833
Processing novel: 27228
Processing novel: 17581
Processing novel: 29559
Processing novel: 28829
Processing novel: 27806
Processing novel: 6363
Processing novel: 15593
Processing novel: 16345
Processing novel: 30092
Processing novel: 15752
Processing novel: 25595
Processing novel: 25523


In [5]:
df = pd.DataFrame(novel_info)
df

,id,name,assoc_names,original_langauge,authors,genres,tags,start_year,licensed,original_publisher,english_publisher,complete_original,chapters_original_current,complete_translated,chapter_latest_translated,release_freq,activity_week_rank,activity_month_rank,activity_all_time_rank,on_reading_lists,reading_list_month_rank,reading_list_all_time_rank,rating,rating_votes,related_series_ids,recommended_series_ids,recommendation_list_ids
0,27232,A Fascinating Marriage with the Colonel,[军婚迷情：老公步步紧逼],chinese,"[an ge, 安歌]",[romance],"[cold love interests, female protagonist, marr...",2020,True,zongheng,tapread,True,346 chapters,False,c40,2.8,2722,2899,4494,256,4100,5194,3.2,12,None,None,"[8137, 4455]"
1,29224,A Love for Three Lives：Monster King Don’t Forg...,[三世情深：魔君，勿忘我],chinese,"[tang hua mo ge, 唐婳墨歌]","[fantasy, romance]","[fallen nobility, fantasy world, female protag...",2019,True,zongheng,tapread,False,444 chapters,False,c11,2.3,2206,2087,6079,160,3757,5808,3.2,9,None,None,None
2,2765,A Record of a Mortal’s Journey to Immortality,"[Fan Ren Xiu Xian Chuan, Phàm Nhân Tu Tiên, RM...",chinese,"[wang yu, 忘语]","[action, adventure, martial arts, xianxia]","[alchemy, androgynous characters, appearance d...",2008,True,qidian,wuxiaworld,True,2446 chapters,False,c1446,0.5,61,52,82,8874,55,189,4.0,643,"[180, 4745]","[15, 180, 181, 5844, 364, 4052]","[7562, 7503, 6922, 6837, 6333]"
3,9120,A Round Trip To Love,"[双程 1 - 归途 Long Way Home, 双程 2 - 殊途 Separate R...",chinese,"[lan lin, 藍淋]","[adult, drama, romance, tragedy, yaoi]","[adapted to movie, incest, long separations, r...",2016,None,None,None,True,6 volumes,False,v3c8,1.5,1713,2042,3346,788,1646,3414,3.5,31,"[1611, 17765, 25382]","[540, 11330, 10025, 13034, 606, 1916]","[8974, 8501]"
4,11764,Adorable Creature Attacks! Cuteness Overload!,"[ACA, Adorable Creature Attack! Beauty is Irre...",chinese,"[ying lanmeng, 盈蓝梦]","[adventure, comedy, drama, fantasy, historical...","[beautiful female lead, female protagonist, ha...",2017,True,zongheng,volare novels,True,197 chapters,False,c85,0.7,1062,1039,1589,2345,1043,1627,4.0,172,None,"[17397, 11567, 4830, 10881, 7094, 3492]","[7199, 7064, 3458, 936]"
5,14233,Adorable Treasured Fox: Divine Doctor Mother O...,"[医后倾天, 爆萌狐宝：神医娘亲要逆天]",chinese,"[xiao qi ye, 萧七爷]","[comedy, fantasy, josei, martial arts, romance...","[alchemy, arrogant characters, beautiful femal...",2017,None,qidian,None,True,2563 chapters,False,c1229,0.5,368,397,138,5999,348,439,4.1,282,None,"[15005, 8414, 13422, 20844, 5865, 16627]","[9017, 8861, 7612, 7256, 7199]"
6,29057,"After Our Divorce, I Still Wore Your Jacket",[离婚之后我还穿着你的外套],chinese,"[not asking if it's three or nine, 不问三九]","[romance, yaoi]","[celebrities, character growth, divorce, moder...",2018,None,jjwxc,None,True,70 chapters,False,c20,1.8,628,685,3723,616,738,3851,4.0,28,[26145],"[19898, 19387, 19498, 19748, 28918, 21253]","[8740, 8260, 8160, 8021, 7355]"
7,14,Against the Gods,"[ATG, Heaven-Defying Evil God, Nghịch Thiên Tà...",chinese,"[mars gravity, 火星引力]","[action, adventure, ecchi, fantasy, harem, mar...","[adapted to manhua, adopted protagonist, arran...",2014,True,zongheng,wuxiaworld,False,1711 chapters,False,c1689,0.9,7,10,5,21270,9,6,3.9,3252,None,"[10, 2072, 791, 6363, 179, 4010]","[9377, 9018, 8890, 8884, 8192]"
8,30064,Asteroid,[小行星],chinese,"[wei feng ji xv, 微风几许]","[comedy, drama, romance, shounen ai, yaoi]","[amnesia, showbiz]",2019,None,jjwxc,None,False,87 chapters,False,c5,0.8,2000,1600,6173,201,3890,5521,4.0,6,None,None,[9139]
9,29462,Bix Magic Cube Game World,[比克斯魔方],chinese,[sisimo],"[adventure, fantasy, romance, sci-fi, shounen ...","[game elements, handsome male lead, master-ser...",2019,None,jjwxc,None,True,143 chapters,False,c19,1.0,1402,810,4893,280,1743,5070,4.2,15,None,[20603],[8740]


In [20]:
df.to_csv('novels.csv', header=True, index=False)